<img src="images/splash3.png">
Requests data for the past hour and inserts this into SQL. To be ran hourly.

## Setup

In [3]:
import sqlite3 as lite
import sys
import coinmarketcap
import requests
import numpy as np
import pandas as pd
import datetime
import matplotlib
import matplotlib.pyplot as plt

## Find tracked coins
Checks sql db for table names

In [4]:
con = None
con = lite.connect('crypto.db')

#Read SQL    
with con:        
    cur = con.cursor()    
    #cur.execute("SELECT * FROM eth")
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")


    
coinlist=[]
for rows in cur:
    coinlist.append(rows[0])
#print(coinlist) #Debug

## Fetch data

In [5]:
def fetch_coindata(symbol, comparison_symbol, limit, aggregate):    
    url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)       
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['mid'] = df[["high", "low"]].mean(axis=1)
    df['timestamp'] = [datetime.datetime.fromtimestamp(d).strftime('%d-%m-%Y %H:%M:%S') for d in df.time]
    return df

## Store values to database

In [6]:
def write_to_db():
    #Connect to DB
    con = None
    con = lite.connect('crypto.db')

    #Dump to SQL
    with con:
        cur = con.cursor()    
        cur.execute("SELECT time FROM XEM ORDER BY time DESC LIMIT 1") # Get the latest
        rows = cur.fetchall()
        latest_db_entry = list(rows[0])
        latest_db_entry = int(latest_db_entry[0])
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coinlist[coin], con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coinlist[coin]) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])        

In [7]:
for coin in range(len(coinlist)):
    try:
        market_data = fetch_coindata(coinlist[coin],'BTC','1','1') # Check what coins are tracked
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        write_to_db()
    except Exception as e:
        print("Skipping coin", coinlist[coin], ", Error: ")
        print(e)
        